#Установка зависимостей

In [ ]:
!pip install -q sentence-transformers faiss-cpu

from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json


from collections import defaultdict
from sentence_transformers import SentenceTransformer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 119.8 MB/s eta 0:00:00


#Ссылки на пути для

In [ ]:
# Пути к файлам
json_path = '/content/merged_ordered_total.json'
out_emb_path = '/content/embeddings_total.npy'
out_meta_path = '/content/metadata_total.json'

##Код для создания эмбеддингов

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import json
from tqdm import tqdm

# Загружаем модель
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')


# Загружаем JSON
with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

vectors = []
meta = []

processed = 0
skipped = 0

print(f'🔄 Обработка {len(data)} записей...')

for item in tqdm(data):
    item_id = item.get("id")
    filename = item.get("filename")

    for field in ['description', 'transcription']:
        text = item.get(field)
        if text:
            emb = model.encode(text)
            vectors.append(emb)
            meta.append({
                'id': item_id,
                'filename': filename,
                'field': field
            })
            processed += 1
        else:
            skipped += 1

# Преобразуем в numpy-массив
vectors = np.vstack(vectors)

# Сохраняем
np.save(out_emb_path, vectors)
with open(out_meta_path, 'w', encoding='utf-8') as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)

print('\n✅ Готово!')
print(f'✔️ Эмбендингов сохранено: {processed}')
print(f'⚠️ Пропущено пустых полей: {skipped}')
print(f'📁 Массив сохранён в: {out_emb_path}')
print(f'📄 Метаданные сохранены в: {out_meta_path}')


🔄 Обработка 10040 записей...


100%|██████████| 10040/10040 [34:51<00:00,  4.80it/s]



✅ Готово!
✔️ Эмбендингов сохранено: 20080
⚠️ Пропущено пустых полей: 0
📁 Массив сохранён в: /content/embeddings_total.npy
📄 Метаданные сохранены в: /content/metadata_total.json


#Ссылки на пути для поиска

In [ ]:
# Пути
embeddings_path = '/content/embeddings_total.npy'
metadata_path = '/content/metadata_total.json'
source_json_path = '/content/merged_ordered_total.json'

## Поисковый алогоритм

In [ ]:
from sentence_transformers import SentenceTransformer
from collections import defaultdict # Import defaultdict here
import numpy as np
import json
import faiss # Also ensure faiss is imported here if used in this cell's functions


def load_data(embeddings_path, metadata_path, source_json_path):
    vectors = np.load(embeddings_path)
    with open(metadata_path, 'r', encoding='utf-8') as f:
        meta = json.load(f)
    with open(source_json_path, 'r', encoding='utf-8') as f:
        source_data = json.load(f)
    return vectors, meta, source_data

def normalize_vectors(vectors):
    return vectors / np.linalg.norm(vectors, axis=1, keepdims=True)

def build_faiss_index(vectors):
    dim = vectors.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(vectors)
    return index

def search_description(query_vector, vecs_norm, meta, source_by_filename, top_k=10):
    desc_idxs = [i for i, m in enumerate(meta) if m['field'] == 'description']
    desc_vectors = vecs_norm[desc_idxs]
    index = build_faiss_index(desc_vectors)
    scores, top_ids = index.search(query_vector, top_k)
    results = []
    for score, idx in zip(scores[0], top_ids[0]):
        real_idx = desc_idxs[idx]
        m = meta[real_idx]
        description = source_by_filename[m['filename']].get('description', '[нет description]')
        results.append({
            'score': float(score),
            'id': m['id'],
            'filename': m['filename'],
            'description': description
        })
    return results

def search_combined(query_vector, vecs_norm, meta, source_by_filename, top_k=10):
    grouped = defaultdict(dict)
    for i, m in enumerate(meta):
        grouped[m['filename']][m['field']] = i

    combined_vectors = []
    combined_meta = []

    for filename, fields in grouped.items():
        if 'description' in fields and 'transcription' in fields:
            i1 = fields['description']
            i2 = fields['transcription']
            combined = vecs_norm[i1] + vecs_norm[i2]
            combined /= np.linalg.norm(combined)
            combined_vectors.append(combined)
            combined_meta.append({
                'filename': filename,
                'id': meta[i1]['id']
            })

    combined_vectors = np.vstack(combined_vectors)
    index = build_faiss_index(combined_vectors)
    scores, top_ids = index.search(query_vector, top_k)

    results = []
    for score, idx in zip(scores[0], top_ids[0]):
        m = combined_meta[idx]
        description = source_by_filename[m['filename']].get('description', '[нет description]')
        results.append({
            'score': float(score),
            'id': m['id'],
            'filename': m['filename'],
            'description': description
        })
    return results

def search_transcription(query_vector, vecs_norm, meta, source_by_filename, top_k=10):
    trans_idxs = [i for i, m in enumerate(meta) if m['field'] == 'transcription']
    trans_vectors = vecs_norm[trans_idxs]
    index = build_faiss_index(trans_vectors)
    scores, top_ids = index.search(query_vector, top_k)
    results = []
    for score, idx in zip(scores[0], top_ids[0]):
        real_idx = trans_idxs[idx]
        m = meta[real_idx]
        description = source_by_filename[m['filename']].get('description', '[нет description]')
        transcription = source_by_filename[m['filename']].get('transcription', '[нет transcription]')
        results.append({
            'score': float(score),
            'id': m['id'],
            'filename': m['filename'],
            'description': description,
            'transcription': transcription
        })
    return results

def search_combined_scores(query_vector, vecs_norm, meta, source_by_filename, top_k=10, threshold=0.5):
    """
    Выполняет комбинированный поиск, вычисляя отдельные показатели для описания и транскрипции,
    суммируя их, применяя пороговое значение и возвращая топ-k результатов.

    Аргументы:
        query_vector: Вектор эмбеддинга запроса.
        vecs_norm: Нормализованные векторы эмбеддингов для всех полей видео.
        meta: Метаданные, связывающие эмбеддинги с именами файлов и полями.
        source_by_filename: Словарь, связывающий имена файлов с их метаданными (описание, id и т.д.).
        top_k: Количество возвращаемых результатов (по умолчанию: 10).
        threshold: Минимальный комбинированный показатель для включения результата (по умолчанию: 0.5).

    Возвращает:
        Список словарей с именем файла, комбинированным показателем, индивидуальными показателями,
        описанием, транскрипцией и id, отсортированный по комбинированному показателю.
    """
    # Шаг 1: Выполняем отдельные поиски для описания и транскрипции
    desc_results = search_description(query_vector, vecs_norm, meta, source_by_filename, top_k=1000)
    trans_results = search_transcription(query_vector, vecs_norm, meta, source_by_filename, top_k=1000)

    # Шаг 2: Собираем показатели для каждого видео в словарь
    video_scores = {}

    # Обрабатываем результаты описания
    for result in desc_results:
        filename = result['filename']
        video_scores[filename] = video_scores.get(filename, {})
        video_scores[filename]['description'] = result['score']

    # Обрабатываем результаты транскрипции
    for result in trans_results:
        filename = result['filename']
        video_scores[filename] = video_scores.get(filename, {})
        video_scores[filename]['transcription'] = result['score']

    # Шаг 3: Вычисляем комбинированные показатели и фильтруем по порогу
    combined_results = []
    for filename, scores in video_scores.items():
        score_desc = scores.get('description', 0)  # По умолчанию 0, если поле отсутствует
        score_trans = scores.get('transcription', 0)  # По умолчанию 0, если поле отсутствует
        score_combined = score_desc + score_trans

        if score_combined >= threshold:
            combined_results.append({
                'filename': filename,
                'score': score_combined,
                'score_description': score_desc,
                'score_transcription': score_trans,
                'description': source_by_filename[filename].get('description', '[Нет описания]'),
                'transcription': source_by_filename[filename].get('transcription', '[Нет транскрипции]'),
                'id': source_by_filename[filename]['id']
            })

    # Шаг 4: Сортируем по комбинированному показателю и возвращаем топ-k
    combined_results.sort(key=lambda x: x['score'], reverse=True)
    return combined_results[:top_k]


# Загрузка данных
vectors, meta, source_data = load_data(embeddings_path, metadata_path, source_json_path)

# Карта для быстрого доступа к описанию
source_by_filename = {item['filename']: item for item in source_data}

# Нормализация векторов
vecs_norm = normalize_vectors(vectors)


#Тестироавние запуска

In [ ]:
# Загрузка модели и получение эмбендинга запроса
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
query = "Школа"
query_vector = model.encode([query], normalize_embeddings=True)


print(f'ЗАПРОС {query}')



# Поиск по сумме description + transcription
comb_results = search_combined(query_vector, vecs_norm, meta, source_by_filename)
print("\n📌 Топ-10 по (description + transcription):")
for r in comb_results:
    print(f"\n🔸 {r['score']:.4f} | ID: {r['id']} | Файл: https://s3.ritm.media/hackaton-itmo/{r['filename']}\n📝 {r['description'][:250]}…")


# Поиск по description
desc_results = search_description(query_vector, vecs_norm, meta, source_by_filename)

print(f'ЗАПРОС {query}')
print("\n📌 Топ-10 по description:")
for r in desc_results:
    print(f"\n🔹 {r['score']:.4f} | ID: {r['id']} | Файл: https://s3.ritm.media/hackaton-itmo/{r['filename']}\n📝 {r['description'][:250]}…")
print(f'ЗАПРОС {query}')
# Поиск по transcription
trans_results = search_transcription(query_vector, vecs_norm, meta, source_by_filename)
print("\n📌 Топ-10 по transcription:")
for r in trans_results:
    print(f"\n🔻 {r['score']:.4f} | ID: {r['id']} | Файл: https://s3.ritm.media/hackaton-itmo/{r['filename']}\n📝{r['description'][:250]}… \n {r['transcription'][:250]}…\n📄 ")
print(f'ЗАПРОС {query}')
# Поиск по сумме score(description) + score(transcription)
comb_results_new = search_combined_scores(query_vector, vecs_norm, meta, source_by_filename, top_k=10, threshold=0.6)
print("\n📌 Топ-10 по (score(description) + score(transcription)) НОВЫЙ ВАРИАНТ С ПОРОГОМ И ДРУГИМ СЛОЖЕНИЕМ:")
for r in comb_results_new:
    print(f"\n🔸 Сумма: {r['score']:.4f} | Описание: {r['score_description']:.4f} | Транскрипция: {r['score_transcription']:.4f} | ID: {r['id']} | Файл: https://s3.ritm.media/hackaton-itmo/{r['filename']}\n📝 Описание: {r['description'][:250]}…\n📜 Транскрипция: {r['transcription'][:250]}…")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

ЗАПРОС Школа

📌 Топ-10 по (description + transcription):

🔸 0.5754 | ID: 1764 | Файл: https://s3.ritm.media/hackaton-itmo/2cc738c8-cd19-4da2-a024-f7ddc422d80e.mp4
📝 The video opens with a close-up of an anime-style character, specifically the upper half of their face and shoulders. The character has dark hair, is wearing glasses, and is dressed in what appears to be a school uniform consisting of a white shirt w…

🔸 0.5527 | ID: 5183 | Файл: https://s3.ritm.media/hackaton-itmo/830ffc39-cee8-40ca-a81c-71aa6740374e.mp4
📝 В начале видео, человек находится в классе и сидит за партой. Затем он начинает рисовать на листке бумаги. После того как он закончил, он поднимает листок и читаёт его вслух.

Далее, человек продолжает писать что-то на другом листке, после чего снова…

🔸 0.5400 | ID: 3988 | Файл: https://s3.ritm.media/hackaton-itmo/6542b03e-79b1-4a06-b265-2c2404cfdf88.mp4
📝 В начале видео, в классическом учебном здании с традиционной атмосферой, студенты и преподаватели сосредоточенно уч

In [ ]:
queries = [
    "Россия",
    "Монако",
    "Blackpink"
]

# Укажите свой путь на диске, например:
path = '/content/queries.json'
with open(path, 'w', encoding='utf-8') as f:
    json.dump(queries, f, ensure_ascii=False, indent=2)

In [ ]:

# Чтение списка запросов
with open('queries.json', 'r', encoding='utf-8') as f:
    queries = json.load(f)

# Инициализация модели эмбеддингов
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Базовый URL для доступа к файлам
source_url_base = 'https://s3.ritm.media/hackaton-itmo/'

# Функции для получения ссылок

def get_combined_links(query: str, top_k: int = 10, threshold: float = 0.6):
    vec = model.encode([query], normalize_embeddings=True)
    results = search_combined_scores(
        vec, vecs_norm, meta, source_by_filename,
        top_k=top_k, threshold=threshold
    )
    return [f"{source_url_base}{r['filename']}" for r in results]



# Собираем результаты по структуре JSON
output = {}
for q in queries:
    output[q] = get_combined_links(q)

# Сохраняем в файл
with open('results.json', 'w', encoding='utf-8') as f:
    json.dump(output, f, ensure_ascii=False, indent=2)

print('Готово! Файл results.json создан с полями combined и description для каждого запроса.')


Готово! Файл results.json создан с полями combined и description для каждого запроса.


In [ ]:
from google.colab import files
files.download('results.json')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>